In [1]:
import pandas as pd
import numpy as np
from time import process_time 
import pickle
import random
from pathlib import Path
import os
import sys
from sklearn import preprocessing
import gc
from scsskutils import *

Found config file! Loading it.
D:\Siddharth Data\PLAsTiCC\Raw Data
Data Location = D:\Siddharth Data\PLAsTiCC\Raw Data
Found pickle location!
Pickle Location = C:\Users\Sid\Desktop\Pickle


In [2]:
data_location

WindowsPath('D:/Siddharth Data/PLAsTiCC/Raw Data')

In [3]:
classes = np.array([6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])
class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1, 99: 1}
pb_wavelength = {0:360, 1:476, 2:621.5, 3:754.5, 4:870, 5:1015}
#https://www.lsst.org/sites/default/files/docs/sciencebook/SB_2.pdf - pg35, Table 2.1

In [4]:
def get_wtable(df):
    all_y = np.array(df['target'], dtype = 'int32')
    y_count = np.unique(all_y, return_counts=True)[1]
    wtable = np.ones(len(classes))
    for i in range(0, y_count.shape[0]):
        wtable[i] = y_count[i] / all_y.shape[0]
    return wtable

In [5]:
target_dict ={
    6: 0,
    15: 1,
    16: 2,
    42: 3,
    52: 4,
    53: 5,
    62: 6,
    64: 7,
    65: 8,
    67: 9,
    88: 10,
    90: 11,
    92: 12,
    95: 13,
    99: 14,
    991: 14,
    992: 14,
    993: 14,
    994: 14
}


# TestSetBatch2

In [6]:
df=pd.read_csv(fr"{data_location}\test_set_batch2.csv")

In [7]:
df2d=pd.read_csv(fr"{data_location}\unblinded_test_set_metadata.csv")

In [8]:
current_objs = df.object_id.unique()
all_objs = df2d.object_id.unique()
indices_to_consider=np.intersect1d(current_objs,all_objs,return_indices=True)[2]
df2d = df2d.iloc[indices_to_consider].reset_index(drop=True)

In [9]:
df2d["target_class"]=df2d.loc[:,["target"]].replace(target_dict)
df2d.drop(["ra","decl","gal_l","gal_b","target"],1,inplace=True)    #Should hostgal_specz be used or not? Hmmm

In [10]:
df =df.merge(df2d[["object_id","hostgal_photoz"]],on="object_id")

In [11]:
df["object_id"] = df["object_id"].astype(np.float64)

In [12]:
df

,object_id,mjd,passband,flux,flux_err,detected,hostgal_photoz
0,1000183.0,59583.2493,0,-4.635374,9.278234,0,0.6753
1,1000183.0,59585.3547,4,6.479336,11.734409,0,0.6753
2,1000183.0,59586.3595,4,-24.911951,13.357832,0,0.6753
3,1000183.0,59590.3573,5,-67.180305,21.674242,0,0.6753
4,1000183.0,59597.1711,5,78.801926,65.485069,0,0.6753
...,...,...,...,...,...,...,...
44281690,13952424.0,60663.2247,3,-3.304427,2.799608,0,0.7949
44281691,13952424.0,60664.2162,2,-6.012751,4.671264,0,0.7949
44281692,13952424.0,60666.1202,1,-0.776305,2.129328,0,0.7949
44281693,13952424.0,60666.2058,2,5.584902,3.180095,0,0.7949


In [13]:
df2d

,object_id,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target_class
0,1000183,0,NaN,0.6753,0.0284,43.0514,0.076,11
1,1000235,0,NaN,0.0000,0.0000,NaN,0.049,12
2,1000260,0,NaN,0.7258,0.0351,43.2431,0.081,3
3,1000280,0,NaN,0.7382,0.0162,43.2881,0.065,11
4,1000282,0,NaN,0.7577,0.0182,43.3577,0.156,10
...,...,...,...,...,...,...,...,...
345992,13952407,0,NaN,0.6187,0.0143,42.8201,0.024,11
345993,13952410,0,NaN,0.3469,0.0387,41.3212,0.139,3
345994,13952415,0,NaN,0.4545,0.1876,42.0133,0.266,11
345995,13952421,0,NaN,0.4323,0.0412,41.8844,0.066,3


In [14]:
df = df.merge(df.loc[:,["object_id","mjd"]].groupby('object_id').first().rename(columns={"mjd":"mjd_start"}),on="object_id")
df["mjd"] = df["mjd"] - df["mjd_start"]
df.drop("mjd_start",1,inplace=True)

In [15]:
# df["mjd"]=df["mjd"]/100

In [16]:
df["mjd"] = df["mjd"]/(df["hostgal_photoz"]+1)

In [17]:
df["lsst_wavelength"]=df.loc[:,["passband"]].replace(pb_wavelength)

In [18]:
df["source_wavelength"]=df["lsst_wavelength"]/(df["hostgal_photoz"]+1)

In [19]:
df.drop("lsst_wavelength",1,inplace=True)

In [ ]:
df["mjd_diff1"]=df.loc[:,["object_id","mjd"]].groupby('object_id').diff(periods=1)
df["mjd_diff2"]=df.loc[:,["object_id","mjd"]].groupby('object_id').diff(periods=-1)
df["mjd_diff1"].fillna(0,inplace=True)
df["mjd_diff2"].fillna(0,inplace=True)

In [ ]:
flux_pow=np.log2(df.loc[:,["object_id","flux"]].groupby('object_id').max() - df.loc[:,["object_id","flux"]].groupby('object_id').min()).reset_index().rename(columns={"flux":"flux_pow"})

In [ ]:
df = df.merge(flux_pow,on="object_id")

In [ ]:
df["flux"]=df["flux"]/np.power(2,df["flux_pow"])
df["flux_err"]=df["flux_err"]/np.power(2,df["flux_pow"])

In [ ]:
flux_pow["small_flux_pow"]=flux_pow["flux_pow"]/10

In [ ]:
df2d = df2d.merge(flux_pow.loc[:,["object_id","small_flux_pow"]],on="object_id")

In [ ]:
assert np.array_equal(df2d.object_id.unique(),df.object_id.unique())

In [ ]:
assert len(df2d) == len(df.object_id.unique())

In [ ]:
df2d.loc[df2d['hostgal_photoz'] > 0, 'nonzero_photoz'] = 1
df2d.loc[df2d['hostgal_photoz'] == 0, 'nonzero_photoz'] = 0

In [ ]:
#band of zerrxy
df["bands"]=df["passband"]+1

In [ ]:
df.drop([
#     "object_id",
    "mjd",
    "passband",
    "detected",           #Decide whether to drop or not
    "hostgal_photoz",
    "flux_pow"],
    1,
    inplace=True)

In [ ]:
labels=df2d.target_class.to_numpy()
df2d.drop("target_class",1,inplace=True)

df2d.drop([
    "object_id",
#     "hostgal_specz",      #Decide whethere to drop or not
],
    1,
    inplace=True)

In [ ]:
df

In [ ]:
df2d.fillna(0,inplace=True)
df.fillna(0,inplace=True)

In [ ]:
#Meta of zerrxy 
twod_data=df2d.to_numpy()

In [ ]:
#Hist of zerrxy, except last column is bands
obj_change_index=np.where(df["object_id"][:-1].to_numpy() != df["object_id"][1:].to_numpy())[0]+1
tuplist=list(zip(np.insert(obj_change_index,0,0),obj_change_index))
dfarray=df.to_numpy()
list_of_data_arrays=[]
for tup in tuplist:
    list_of_data_arrays.append(dfarray[tup[0]:tup[1],1:])
list_of_data_arrays.append(dfarray[obj_change_index[-1]:,1:])
#Then sequence padsequences
#max_length=max(map(len,list_of_data_arrays))
#threed_data=sequence.pad_sequences(list_of_data_arrays, maxlen=max_length)

In [ ]:
df.columns

In [ ]:
df2d.columns

In [ ]:
with open(fr"{pickle_location}\test2_3d_pickle", "wb") as fp:   #Pickling
    pickle.dump(list_of_data_arrays, fp)

with open(fr"{pickle_location}\test2_2d_pickle", "wb") as fp:   #Pickling
    pickle.dump(twod_data, fp)
    
with open(fr"{pickle_location}\test2_label_pickle", "wb") as fp:   #Pickling
    pickle.dump(labels, fp)